In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

my_dpi = 192

In [28]:
# Load data and create dataframe
phi = np.load("Phi_201snap.npy")
df = pd.DataFrame(columns=['norm'], index=np.arange(201))

for i in range(201):
    arr = phi[:,:,i]

    # Plot image (raw automatically converted to grayscale)
    plt.figure(figsize=(294/my_dpi, 165/my_dpi), dpi=my_dpi)
    plt.imshow(arr, cmap='gray')
    plt.axis("off")

    # Save as JPG and open again to convert to np array
    plt.savefig('analysis2.1.jpg', bbox_inches='tight', pad_inches=0)
    plt.close()
    im = Image.open('analysis2.1.jpg')
    output_array = np.asarray(im)

    # Create convert function to use to convert output_array back to raw values
    phimax, phimin = np.max(arr), np.min(arr)
    ratio = (phimax-phimin)/255
    def convertRGBToRaw(x):
        return ratio*x + phimin

    # Loop through output_array and replace RGB by raw in copy
    copy = arr.copy()
    for i, row in enumerate(output_array):
        for j, col in enumerate(row):
            copy[i][j] = convertRGBToRaw(col[0])

    # Compute norm and add to dataframe
    diff = copy - arr
    norm = np.linalg.norm(diff)
    df.at[i, 'norm'] = norm
df

,norm
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
196,NaN
197,NaN
198,NaN
199,NaN


In [16]:
# df['norm'].isnull().sum() # counting total number of NaN's in df
# np.isnan(np.sum(phi)) # returns true if there's at least one NaN in phi, false otherwise
df.at[101, 'norm']

nan

In [43]:
# Computing single norms without for loop
i = 160

my_dpi = 192
phi = np.load("Phi_201snap.npy")
arr = phi[:,:,i] 
plt.figure(figsize=(294/my_dpi, 165/my_dpi), dpi=my_dpi)
plt.imshow(arr, cmap='gray')
plt.axis("off")
plt.savefig('analysis2.1.jpg', bbox_inches='tight', pad_inches=0)
plt.close()
im = Image.open('analysis2.1.jpg')
output_array = np.asarray(im)
output_array
copy = arr.copy()
phimax, phimin = np.max(arr), np.min(arr)
ratio = (phimax-phimin)/255
def convertRGBToRaw(x):
    return ratio*x + phimin
for i, row in enumerate(output_array):
    for j, col in enumerate(row):
        copy[i][j] = convertRGBToRaw(col[0])
norm = np.linalg.norm(copy - arr)
output_array

array([[[121, 121, 121],
        [121, 121, 121],
        [121, 121, 121],
        ...,
        [122, 122, 122],
        [122, 122, 122],
        [122, 122, 122]],

       [[121, 121, 121],
        [121, 121, 121],
        [121, 121, 121],
        ...,
        [121, 121, 121],
        [121, 121, 121],
        [121, 121, 121]],

       [[121, 121, 121],
        [121, 121, 121],
        [121, 121, 121],
        ...,
        [119, 119, 119],
        [119, 119, 119],
        [119, 119, 119]],

       ...,

       [[121, 121, 121],
        [121, 121, 121],
        [121, 121, 121],
        ...,
        [119, 119, 119],
        [119, 119, 119],
        [119, 119, 119]],

       [[120, 120, 120],
        [120, 120, 120],
        [120, 120, 120],
        ...,
        [119, 119, 119],
        [119, 119, 119],
        [119, 119, 119]],

       [[120, 120, 120],
        [120, 120, 120],
        [120, 120, 120],
        ...,
        [119, 119, 119],
        [119, 119, 119],
        [119, 119, 119]]

In [39]:
# Debugging attempt
n = 108

# Load data and create dataframe
phi = np.load("Phi_201snap.npy")
df = pd.DataFrame(columns=['norm'], index=np.arange(n))

for i in range(n):
    arr = phi[:,:,i]

    # Plot image (raw automatically converted to grayscale)
    plt.figure(figsize=(294/my_dpi, 165/my_dpi), dpi=my_dpi)
    plt.imshow(arr, cmap='gray')
    plt.axis("off")

    # Save as JPG and open again to convert to np array
    plt.savefig('analysis2.1.jpg', bbox_inches='tight', pad_inches=0)
    plt.close()
    im = Image.open('analysis2.1.jpg')
    output_array = np.asarray(im)

    # Create convert function to use to convert output_array back to raw values
    phimax, phimin = np.max(arr), np.min(arr)
    ratio = (phimax-phimin)/255
    def convertRGBToRaw(x):
        return ratio*x + phimin

    # Loop through output_array and replace RGB by raw in copy
    copy = arr.copy()
    for i, row in enumerate(output_array):
        for j, col in enumerate(row):
            copy[i][j] = convertRGBToRaw(col[0])

    # Compute norm and add to dataframe
    diff = copy - arr
    norm = np.linalg.norm(diff)
    df.at[i, 'norm'] = norm
df

,norm
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
103,NaN
104,NaN
105,NaN
106,NaN


In [22]:
df['norm'].isnull().sum()

5